# The XGBoost 1 feature regression

## Imports

### Py Imports

In [ ]:
import pandas as pd
import numpy as np
import xgboost
import matplotlib.pyplot as plt
import seaborn as sbn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import pickle

### Datasets import

In [ ]:
train = pd.read_csv('./sources/clean_train.csv')
test = pd.read_csv('./sources/clean_test.csv')

### Load variables

In [ ]:
with open('./columns.pkl', 'rb') as col:
    all_col = pickle.load(col)
    
num_col = all_col['num_col']
cat_col = all_col['cat_col']

## work on dataset

### Scale datas

In [ ]:
Y_train = train['SalePrice'].copy()
X_train = train.drop('SalePrice', axis=1).copy()

In [ ]:
X_train_num = X_train[num_col].copy()

In [ ]:
X_train['Surface'] = X_train['GrLivArea'] + X_train['TotalBsmtSF']
X_train = X_train.drop(['GrLivArea', 'TotalBsmtSF'], axis=1)
X_train